In [1]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.pipeline import merge_entities

In [2]:
nlp = spacy.load("en_core_web_sm")
# patterns = [
#     {"label":"DATE", "pattern":[{"TEXT": {"REGEX": "^([0-2][0-9]|(3)[0-1])(\/)(((0)[0-9])|((1)[0-2]))(\/)\d{4}$"}}]}
#     {"label":"COL-DET", "pattern":}
# ]
# ruler.add_patterns(patterns)
# ruler.to_disk("../data/patterns.jsonl")

ruler = EntityRuler(nlp, overwrite_ents=True).from_disk("../data/patterns.jsonl")

In [3]:
nlp.add_pipe(ruler)
nlp.add_pipe(merge_entities)

In [4]:
inputQs = []
with open('../data/queslist.txt', 'r') as f:
    x = f.readlines()
    for t in range(0,14000,50):
        tx = x[t].strip()
        inputQs.append(tx)
    

In [5]:
# displacy.render(doc, options={'fine-grained': True})

In [6]:
inputQs = ['Who was the winner between Mum Indians and Super Kings at Mumbai?',
 'Who was the loser between Mum Indians and Super Kings at Mumbai?',
 'Who was the winner between Mum Indians and Super Kings on 07/04/2018?',
 'Who was the loser between Mum Indians and Super Kings on 07/04/2018?',
 'When was the match between Mum Indians & Super Kings held at Mumbai?',
 'Where was the match between Mum Indians & Super Kings held on 07/04/2018?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored by RG Sharma in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against DL Chahar in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did DL Chahar bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many runs were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many fours were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings at Mumbai?',
 'How many sixes were scored against SR Watson in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many overs did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many no balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings at Mumbai?',
 'How many wide balls did SR Watson bowl in the match between Mum Indians and Super Kings on 07/04/2018?']

In [25]:
def info(doc):
    displacy.render(doc, style='ent')
    print("{:13} {:10} {:20} {:20} {:20} {:20}".format("token.text", "token.pos_", "token.tag_", "token.dep_", "token.lefts", "token.rights"))
    for token in doc:
        print("{:13} {:10} {:20} {:20} {:20} {:20}".format(token.text, token.pos_, token.tag_, token.dep_, str([token.text for token in token.lefts]), str([token.text for token in token.rights])))
    print()
    print("{:25} {:25} {:25} {:25}".format("chunk.text", "chunk.root.text", "chunk.root.dep_", "chunk.root.head.text"))
    for chunk in doc.noun_chunks:
        print("{:25} {:25} {:25} {:25}".format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))
    # Finding a verb with a subject from below — good
    print()
    print("{:25} {:25}".format("ent.text", "ent.label_"))
    for ent in doc.ents:
        print("{:25} {:25}".format(ent.text, ent.label_))
    print()
    displacy.render(doc)
    return

def getType(doc):
    if doc[0].text == "How":
        return 2
    else:
        return 1

def typeProcessing(doc, qtype):
    if qtype == 1:
        pass
    if qtype == 2:
        ## Added a rule here, if query is of definite type, then merge phrase if left tree contains more than 1 ('many' is always included in the left tree)
        if(doc[3].n_lefts == 2):
            span = doc[doc[3].left_edge.i+2 : doc[3].right_edge.i+1]
            with doc.retokenize() as retokenizer:
                retokenizer.merge(span, attrs={"pos":u"NOUN", "tag":"NNS"})
    return doc

def createQuery(doc):
    w_cl = "WHERE "
    team_flag = 0
    t1 = ""
    t2 = ""
    for ent in doc.ents:
        if w_cl[-1] != " ":
            w_cl += " AND "
        if team_flag == 0 and str(ent.label_) == "TEAM":
            t1 = ent.text
#             t_cond = "(TEAM1" + ' = "' + ent.text + '")'
            team_flag = 1
        elif team_flag == 1 and str(ent.label_) == "TEAM":
            t2 = ent.text
#             t_cond += " AND (TEAM2" + ' = "' + ent.text + '")'
#             w_cl += "("+ t_cond + ")"
            team_flag = 2
        else:
            w_cl += "(" + ent.label_ + ' = "' + ent.text + '")'
    if team_flag == 2:
        t_cond = '(TEAM1 = "{}" AND TEAM2 = "{}")'
        x = t_cond.format(t1,t2)
        y = t_cond.format(t2,t1)
        if w_cl[-1] != " ":
            w_cl += " AND "
        w_cl += "(" + x +" OR " + y + ")"
    print(w_cl)

In [26]:
def Hub(questions):
    for NLquery in questions:
        doc = nlp(NLquery)
#         info(doc)
        qtype = getType(doc)
        doc = typeProcessing(doc, qtype)
        info(doc)
        # Get relavent terms for query formation
        relv = set()
        for token in doc:
            if (token.pos_ == "PRON" or token.pos_ == "NOUN") and token.dep_ == "attr":
                relv.add(token.text)
            if token.pos_ == "ADV":
                relv.add(token.text)
            if token.pos_ == "NOUN" and (token.dep_ == "nsubjpass" or token.dep_ == "nsubj"):
                relv.add(token.text)
            if token.pos_ == "ADP" and (token.dep_ == "agent" or token.dep_ == "prep"):
                relv.add(token.text)
        print(relv)
        
        createQuery(doc)

In [27]:
Hub(inputQs)

token.text    token.pos_ token.tag_           token.dep_           token.lefts          token.rights        
Who           PRON       WP                   attr                 []                   []                  
was           VERB       VBD                  ROOT                 ['Who']              ['winner', '?']     
the           DET        DT                   det                  []                   []                  
winner        NOUN       NN                   attr                 ['the']              ['between', 'at']   
between       ADP        IN                   prep                 []                   ['Mum Indians']     
Mum Indians   PROPN      NNPS                 pobj                 []                   ['and', 'Super Kings']
and           CCONJ      CC                   cc                   []                   []                  
Super Kings   NOUN       NNS                  conj                 []                   []                  
at            ADP

{'Who', 'at', 'winner', 'between'}
WHERE (GROUND = "Mumbai") AND ((TEAM1 = "Mum Indians" AND TEAM2 = "Super Kings") OR (TEAM1 = "Super Kings" AND TEAM2 = "Mum Indians"))


token.text    token.pos_ token.tag_           token.dep_           token.lefts          token.rights        
Who           PRON       WP                   attr                 []                   []                  
was           VERB       VBD                  ROOT                 ['Who']              ['loser', '?']      
the           DET        DT                   det                  []                   []                  
loser         NOUN       NN                   attr                 ['the']              ['between', 'at']   
between       ADP        IN                   prep                 []                   ['Mum Indians']     
Mum Indians   PROPN      NNPS                 pobj                 []                   ['and', 'Super Kings']
and           CCONJ      CC                   cc                   []                   []                  
Super Kings   NOUN       NNS                  conj                 []                   []                  
at            ADP

{'Who', 'at', 'loser', 'between'}
WHERE (GROUND = "Mumbai") AND ((TEAM1 = "Mum Indians" AND TEAM2 = "Super Kings") OR (TEAM1 = "Super Kings" AND TEAM2 = "Mum Indians"))


token.text    token.pos_ token.tag_           token.dep_           token.lefts          token.rights        
Who           PRON       WP                   attr                 []                   []                  
was           VERB       VBD                  ROOT                 ['Who']              ['winner', 'on', '07/04/2018', '?']
the           DET        DT                   det                  []                   []                  
winner        NOUN       NN                   attr                 ['the']              ['between']         
between       ADP        IN                   prep                 []                   ['Mum Indians']     
Mum Indians   PROPN      NNPS                 pobj                 []                   ['and', 'Super Kings']
and           CCONJ      CC                   cc                   []                   []                  
Super Kings   NOUN       NNS                  conj                 []                   []                  
on

...

{'on', 'between', 'How', 'in', 'no balls'}
WHERE (PLAYER = "SR Watson") AND (DATE = "07/04/2018") AND ((TEAM1 = "Mum Indians" AND TEAM2 = "Super Kings") OR (TEAM1 = "Super Kings" AND TEAM2 = "Mum Indians"))


token.text    token.pos_ token.tag_           token.dep_           token.lefts          token.rights        
How           ADV        WRB                  advmod               []                   []                  
many          ADJ        JJ                   amod                 ['How']              []                  
wide balls    NOUN       NNS                  nsubj                ['many']             []                  
did           VERB       VBD                  ROOT                 ['wide balls']       ['bowl', '?']       
SR Watson     PROPN      NNP                  compound             []                   []                  
bowl          NOUN       NN                   dobj                 ['SR Watson']        ['in']              
in            ADP        IN                   prep                 []                   ['match']           
the           DET        DT                   det                  []                   []                  
match         NOUN 

{'between', 'How', 'wide balls', 'in', 'at'}
WHERE (PLAYER = "SR Watson") AND (GROUND = "Mumbai") AND ((TEAM1 = "Mum Indians" AND TEAM2 = "Super Kings") OR (TEAM1 = "Super Kings" AND TEAM2 = "Mum Indians"))


token.text    token.pos_ token.tag_           token.dep_           token.lefts          token.rights        
How           ADV        WRB                  advmod               []                   []                  
many          ADJ        JJ                   amod                 ['How']              []                  
wide balls    NOUN       NNS                  nsubj                ['many']             []                  
did           VERB       VBD                  ROOT                 ['wide balls']       ['bowl', 'on', '07/04/2018', '?']
SR Watson     PROPN      NNP                  compound             []                   []                  
bowl          NOUN       NN                   dobj                 ['SR Watson']        ['in']              
in            ADP        IN                   prep                 []                   ['match']           
the           DET        DT                   det                  []                   []                  
match 

{'between', 'How', 'wide balls', 'in', 'on'}
WHERE (PLAYER = "SR Watson") AND (DATE = "07/04/2018") AND ((TEAM1 = "Mum Indians" AND TEAM2 = "Super Kings") OR (TEAM1 = "Super Kings" AND TEAM2 = "Mum Indians"))
